In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import os
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
dfmovies = pd.read_csv('../data/rotten_tomatoes_movies.csv')
dfcritics = pd.read_csv('../data/rotten_tomatoes_critic_reviews.csv')

In [14]:
dfmovies.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,Movie,Title
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,49.0,149.0,Spilled,53.0,254421.0,43,73,76,alway troublepron life teenag perci jackson lo...,perci jackson olympian lightn thief
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,87.0,142.0,Upright,64.0,11574.0,44,123,19,kate catherin keener husband alex oliv platt w...,pleas give
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,67.0,24.0,Spilled,53.0,14684.0,2,16,8,success middleag hollywood songwrit fall hopel...,10
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,100.0,54.0,Upright,97.0,105386.0,6,54,0,follow close argument murder trial 12 member j...,12 angri men twelv angri men
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,89.0,27.0,Upright,74.0,68918.0,5,24,3,1866 professor pierr aronnax paul luka assist ...,20000 leagu sea


In [12]:
dfcritics.head()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [18]:
dfcritics['review_content'] = dfcritics['review_content'].fillna("").astype(str)

In [19]:
def clean_text(text):
    """Elimina caracteres no deseados y normaliza el texto."""
    text = text.lower()  # Convertir a minúsculas
    text = text.translate(str.maketrans('', '', string.punctuation))  # Eliminar puntuación
    text = text.strip()  # Eliminar espacios iniciales y finales
    return text

In [20]:
def preprocess_text(content):
    """Realiza la limpieza, tokenización, eliminación de stopwords y stemming del texto."""
    # Limpieza de texto
    cleaned_text = clean_text(content)
    
    # Tokenizar documentos
    tokens = word_tokenize(cleaned_text)
    
    # Eliminación de stopwords
    stop_words = set(stopwords.words('english'))

    # Tokenización
    tokens = [word for word in tokens if word not in stop_words]
    
    # Aplicación de stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]

    # Normalización


    # Reconstrucción del texto preprocesado
    preprocessed_text = " ".join(stemmed_tokens)
    return preprocessed_text

In [21]:
def preprocess_documents(data):
    data['Review'] = data['review_content'].apply(preprocess_text)
    return data


In [22]:
df = preprocess_documents(dfcritics)

In [92]:
df.head()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content,Review
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...,fantasi adventur fuse greek mytholog contempor...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff...",uma thurman medusa gorgon coiffur writh snake ...
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...,topnotch cast dazzl special effect tide teen n...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...,whether audienc get behind lightn thief hard p...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...,what realli lack lightn thief genuin sens wond...


In [24]:
movies_dir = r'..\data'
output_excel_path = os.path.join(movies_dir, "critics_data_preprocessed.xlsx")
df.to_excel(output_excel_path, index=False)

ValueError: This sheet is too large! Your sheet size is: 1130017, 9 Max sheet size is: 1048576, 16384

In [93]:
def cargar_datos_desde_excel(df):

    if not {'rotten_tomatoes_link', 'Review'}.issubset(df.columns):
        raise ValueError("El archivo Excel debe el id y la review")
    documentos = df.to_dict(orient='records')  # Convertir DataFrame a lista de diccionarios
    return documentos

In [94]:
# Cargar datos desde Excel
documentos = cargar_datos_desde_excel(df)

In [95]:
def procesar_consulta_local_y_unir(query, dfmovies):
    """Procesar consulta, devolver los 10 mejores resultados y hacer un inner join con dfmovies."""
    if not query:
        return {"error": "La consulta está vacía"}

    # Vectorizar la consulta con TF-IDF
    query_vector = tfidf_vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Seleccionar los 10 resultados con mayor similitud
    ranked_indices = similarities.argsort()[::-1][:10]  # Solo los 10 primeros

    # Preparar resultados con ranking
    results = []
    for rank, idx in enumerate(ranked_indices, start=1):
        results.append({
            "Ranking": rank,
            "rotten_tomatoes_link": documentos[idx]['rotten_tomatoes_link'],
            "Review": documentos[idx]['Review'],
            "Similaridad": round(similarities[idx], 4)
        })

    # Convertir los resultados a un dataframe
    df_results = pd.DataFrame(results)

    # Realizar el inner join con dfmovies
    df_joined = pd.merge(df_results, dfmovies, on='rotten_tomatoes_link', how='inner')

    # Convertir los resultados combinados en una lista de diccionarios
    combined_results = df_joined.to_dict(orient='records')
    return combined_results

In [97]:
# Ejemplo de uso local
if __name__ == "__main__":
    consulta = input("Introduce tu consulta: ")

    resultados = procesar_consulta_local_y_unir(consulta, dfmovies)

    if "error" in resultados:
        print("Error:", resultados["error"])
    else:
        print("\nResultados Combinados:")
        for resultado in resultados:
            print(f"Título: {resultado['movie_title']}")
            print(f"Similaridad: {resultado['Similaridad']}\n")

Introduce tu consulta:  space



Resultados Combinados:
Título: Ad Astra
Similaridad: 0.7434

Título: The Ladies Man
Similaridad: 0.7135

Título: Moonraker
Similaridad: 0.7115

Título: L'Eclisse
Similaridad: 0.6649

Título: Passengers
Similaridad: 0.6262

Título: The Jetsons
Similaridad: 0.6188

Título: Hunter Killer
Similaridad: 0.6087

Título: Space Chimps
Similaridad: 0.6055

Título: The French Connection II
Similaridad: 0.6047

Título: Gravity
Similaridad: 0.6046

